In [1]:
import random
import numpy as np
import math
import time
import os

In [2]:
def euclidean_distance(a, b):
    return math.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

In [3]:
def genetic_backpack(max_capacity, w, c):
    N = len(w)
    n_chromo = 200
    dna = np.zeros(N*n_chromo).reshape(n_chromo, N)
    k = 2
    # генерируем хромосомы
    for i in range(n_chromo):
        for j in range(N):
            dna[i][j] = random.randint(0,1)
    #print('Популяция:\n', dna)
    def remove_random_item(chromo):
        one_positions = []
        for i in range(len(chromo)):
            if chromo[i]:
                one_positions.append(i)
        chromo[one_positions[random.randint(0, len(one_positions) - 1)]] = 0
    
    best_solution = []
    for i in range(100):
        # Считаем вес и стоимость
        def fitness_function(chromo, w, c):
            temp_w, temp_p = 0, 0
            for j in range(N):
                if chromo[j]:
                    temp_w += w[j]
                    temp_p += c[j]
            return temp_p, temp_w
        total_chromo = []
        for i in range(n_chromo):
            temp_p, temp_w = fitness_function(dna[i], w, c)

            while temp_w > max_capacity:
                remove_random_item(dna[i])
                temp_p, temp_w = fitness_function(dna[i], w, c)
            total_chromo.append((i, temp_p, temp_w))
        #print('Популяция после подсчета фит. функции:\n', dna)  

        # Сортируем
        total_chromo = sorted(total_chromo, key=lambda x: x[1], reverse=True)
        #print(total_chromo)
        best_solution = [dna[total_chromo[0][0]], total_chromo[0][1], total_chromo[0][2]]
        
        # Находим распределение вероятностей
        fitness_sum = 0
        for x in total_chromo:
            fitness_sum += x[1]
        distribution = []
        for chromo in dna:
            distribution.append(fitness_function(chromo, w, c)[0] / fitness_sum)
        d = n_chromo // 10
        for i in range(n_chromo - d, n_chromo):
            distribution[i] *= 2
        #distribution.sort(reverse=True)
        #print(sorted(distribution), reverse=True)
        # выбираем для селекции
        chromos_selected = random.choices(dna, weights=distribution, k=k)
        # чтобы не брать одинаковые
        while len(np.unique(chromos_selected, axis=0)) < k:
            chromos_selected = random.choices(dna, weights=distribution, k=k)
        #print('Выбрали для селекции:\n',chromos_selected)
        
        indices = []
        for i in range(len(dna)):
            if (dna[i] == chromos_selected[0]).all() or (dna[i] == chromos_selected[1]).all():
                indices.append(i)
        dna = np.delete(dna, [indices[0], indices[1]], 0)

        # делаем кроссовер
        crossover_point = random.randint(1, N-1)
        tmp = chromos_selected[1][:crossover_point].copy()
        chromos_selected[1][:crossover_point], chromos_selected[0][:crossover_point] = chromos_selected[0][:crossover_point], tmp
        #print('После селекции:\n', chromos_selected)

        # мутация
        for j in range(len(chromos_selected)):
            for i in range(len(chromos_selected[j])):
                if random.randint(1, 50) == 1:
                    chromos_selected[j][i] = (chromos_selected[j][i] + 1) % 2
        #print('После мутации:\n', chromos_selected)

        # считаем ф-ф полученных хромо, выравниваем веса
        fitness_results_selected = []
        for i in range(len(chromos_selected)):
            temp_p, temp_w = fitness_function(chromos_selected[i], w, c)
            while temp_w > max_capacity:
                remove_random_item(chromos_selected[i])
                temp_p, temp_w = fitness_function(chromos_selected[i], w, c)
            fitness_results_selected.append((i, temp_p, temp_w))

        dna = np.append(dna, [chromos_selected[0]], 0)
        dna = np.append(dna, [chromos_selected[1]], 0)

        fitness_results_selected.sort(key=lambda x: x[1], reverse=True)

        if fitness_results_selected[0][1] > best_solution[2]: #total_chromo[0][1]:
            best_solution = [chromos_selected[fitness_results_selected[0][0]], fitness_results_selected[0][1], fitness_results_selected[0][2]]  #fitness_results_selected[0]] #(chromos_selected[fitness_results_selected[0][0]], fitness_results_selected[0][1], fitness_results_selected[0][2])
        #else:
            #best_solution = dna[total_chromo[0][0]], total_chromo[0][1], total_chromo[0][2]
    return best_solution

In [13]:
p_c = []
p_p = []
p_s = []
p_w = []
for i in range(1, 8):
    p_c.append(open(os.getcwd() + "\\benchmarks_knapsack\\p0" + str(i) + "_c.txt", "r"))
    p_p.append(open(os.getcwd() + "\\benchmarks_knapsack\\p0" + str(i) + "_p.txt", "r"))
    p_s.append(open(os.getcwd() + "\\benchmarks_knapsack\\p0" + str(i) + "_s.txt", "r"))
    p_w.append(open(os.getcwd() + "\\benchmarks_knapsack\\p0" + str(i) + "_w.txt", "r"))
for test_num in range(7):
    print(f"BENCHMARK p0{test_num+1}")
    w = int(p_c[test_num].read())
    items_c = []
    for line in p_p[test_num]:
        items_c.append(int(line))
        
    items_w = []
    for line in p_w[test_num]:
         items_w.append(int(line))

    items_true = []
    w_true = 0
    c_true = 0
    for line in p_s[test_num]:
        items_true.append(int(line))
    for j in range(len(items_true)):
        if items_true[j] == 1:         
            w_true += items_w[j]
            c_true += items_c[j]
                
    answers = []
    timeExecuted = 0
    for i in range(100):
        start = time.time()
        answers.append(genetic_backpack(w, items_w, items_c))
        timeExecuted += time.time() - start
    #answers.append(genetic_backpack(w, items_w, items_c))
    print(f"TIME: {timeExecuted/100*1000}")
    bestResult = max(list(map(lambda x: x[1], answers)))
    print(f'COST: {bestResult}')
    for i in answers:
        if i[1] == bestResult:
            print(f'ITEMS: {i[0]}')
            break

BENCHMARK p01
TIME: 184.08037185668945
COST: 309
ITEMS: [1. 1. 1. 1. 0. 1. 0. 0. 0. 0.]
BENCHMARK p02
TIME: 152.61001110076904
COST: 51
ITEMS: [0. 1. 1. 1. 0.]
BENCHMARK p03
TIME: 159.9299955368042
COST: 150
ITEMS: [1. 1. 0. 0. 1. 0.]
BENCHMARK p04
TIME: 167.260000705719
COST: 107
ITEMS: [1. 0. 0. 1. 0. 0. 0.]
BENCHMARK p05
TIME: 174.54999923706055
COST: 900
ITEMS: [1. 0. 1. 1. 1. 0. 1. 1.]
BENCHMARK p06
TIME: 166.07998609542847
COST: 1735
ITEMS: [0. 1. 0. 1. 0. 0. 1.]
BENCHMARK p07
TIME: 228.30999612808228
COST: 1436
ITEMS: [1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1.]


In [45]:
def genetic_salesman(graph):
    n_edges = len(graph)
    n_chromo = 100
    k = 2
    population = []
    
    for i in range(n_chromo):
        population.append([i for i in range(n_edges)])
        random.shuffle(population[i])
    
    def fitness_function(chromo):
        res = 0
        for i in range(n_edges-1):
            res += graph[chromo[i]][chromo[i+1]]
        return res
    
    fitness_values = []
    fitness_sum = 0
    for i in range(n_chromo):
        ff = fitness_function(population[i])
        fitness_values.append((i, ff))
        fitness_sum += ff
    fitness_values.sort(key=lambda x: x[1])
    
    distribution = [0 for i in range(n_chromo)]
    for x in fitness_values:
        distribution[x[0]] = x[1] / fitness_sum
            
    chromos_selected = random.choices(population, weights=distribution, k=k)
    while len(np.unique(chromos_selected, axis=0)) < k:
        chromos_selected = random.choices(population, weights=distribution, k=k)
    
    # crossover
        
    sep = random.choices([i for i in range(n_edges)], k=2)
    while len(np.unique(sep, axis=0)) < k:
        sep = random.choices([i for i in range(n_edges)], k=2)
    sep.sort()
    chromos_crossovered = [[-1 for i in range(n_edges)], [-1 for i in range(n_edges)]]
    print(sep[0], sep[1])
    pairs = []
    collisions = [[],[]]
    for i in range(sep[0], sep[1]):
        pairs.append((chromos_selected[0][i], chromos_selected[1][i]))
        chromos_crossovered[0][i] = chromos_selected[1][i]
        chromos_crossovered[1][i] = chromos_selected[0][i]
        collisions[0].append(chromos_selected[1][i])
        collisions[1].append(chromos_selected[0][i])
    print(chromos_crossovered)
    for i in range(n_edges):
        if chromos_selected[0][i] not in collisions[0]:
            chromos_crossovered[0][i] = chromos_selected[0][i]
    print(chromos_crossovered)
    for i in range(n_edges):
        if chromos_selected[0][i] in collisions[0] and chromos_crossovered[0][i] == -1:
            temp = chromos_selected[0][i]
            while temp in collisions[0]:
                for p in pairs:
                    if temp == p[0] and p[1] not in collisions[0]:
                        temp = p[1]
                        print(f'{p[0]} changed to {p[1]}')
                        collisions[0].append(p[0])
                        #break
                    elif temp == p[1] and p[0] not in collisions[0]:
                        temp = p[0]
                        print(f'{p[1]} changed to {p[0]}')
                        collisions[0].append(p[1])
                        #break
            chromos_crossovered[0][i] = temp
            
        if chromos_selected[1][i] in collisions[1] and chromos_crossovered[1][i] == -1:
            temp = chromos_selected[1][i]
            while temp in collisions[1]:
                for p in pairs:
                    if temp == p[0] and p[1] not in collisions[1]:
                        #prev_temp = p[0]
                        collisions[1].append(p[0])
                        temp = p[1]
                        #break
                    elif temp == p[1] and p[0] not in collisions[1]:
                        temp = p[0]
                        collisions[1].append(p[1])
                        #break
            chromos_crossovered[0][i] = temp
        
    return sep, chromos_selected, chromos_crossovered

In [4]:
def fitness_function(chromosome, edges_weight):
    ans = 0
    for i in range(len(chromosome)-1):
        ans += edges_weight[chromosome[i]][chromosome[i+1]]
    return ans


def get_ans(edges_weight):
    num_of_edges = len(edges_weight)
    num_of_chromosomes = 1000
    k = num_of_chromosomes // 4
    k = k // 4
    k *= 4
    chromosomes = []
    for i in range(num_of_chromosomes):
        chromosomes.append([])
        for j in range(num_of_edges):
            chromosomes[i].append(j)
        random.shuffle(chromosomes[i])

    fitness_results = []
    for i in range(num_of_chromosomes):
        tmp = fitness_function(chromosomes[i], edges_weight)
        fitness_results.append((i, tmp))
    fitness_results.sort(key=lambda x: x[1])

    random_list = [0 for i in range(num_of_chromosomes)]
    for i in range(k):
        random_list[i] = 1
    random.shuffle(random_list)

    chromosomes_selected = []
    for i in range(num_of_chromosomes):
        if random_list[i] == 1:
            chromosomes_selected.append((i, fitness_results[i]))

    chromosomes_selected.sort(key=lambda x: x[1])
    chromosomes_selected = chromosomes_selected[:len(chromosomes_selected)//2]

    for i in range(0, len(chromosomes_selected), 2):

        crossover_point1 = random.randint(0, num_of_edges - 2)

        crossover_point2 = random.randint(crossover_point1+1, num_of_edges)

        child1 = [-1 for i in range(num_of_edges)]

        child2 = [-1 for i in range(num_of_edges)]

        tmp1 = chromosomes[chromosomes_selected[i+1][0]][crossover_point2:] + chromosomes[chromosomes_selected[i+1][0]][:crossover_point1]
        tmp2 = chromosomes[chromosomes_selected[i][0]][crossover_point2:] + chromosomes[chromosomes_selected[i][0]][:crossover_point1]

        for j in range(crossover_point1, crossover_point2):
            child1[j] = chromosomes[chromosomes_selected[i][0]][j]
            child2[j] = chromosomes[chromosomes_selected[i+1][0]][j]
            if chromosomes[chromosomes_selected[i+1][0]][j] in tmp1:
                tmp1.remove(chromosomes[chromosomes_selected[i+1][0]][j])
            if chromosomes[chromosomes_selected[i][0]][j] in tmp2:
                tmp2.remove(chromosomes[chromosomes_selected[i][0]][j])

        child1[crossover_point2:] = tmp1[:num_of_edges - crossover_point2]
        child1[:crossover_point1] = tmp1[num_of_edges - crossover_point2:]

        child2[crossover_point2:] = tmp2[:num_of_edges - crossover_point2]
        child2[:crossover_point1] = tmp2[num_of_edges - crossover_point2:]

        if random.randint(1, 100) == 42:
            pointer1 = random.randint(0, num_of_edges - 1)
            pointer2 = random.randint(0, num_of_edges - 1)
            child1[pointer1], child1[pointer2] = child1[pointer2], child1[pointer1]

        if random.randint(1, 100) == 42:
            pointer1 = random.randint(0, num_of_edges - 1)
            pointer2 = random.randint(0, num_of_edges - 1)
            child2[pointer1], child2[pointer2] = child2[pointer2], child2[pointer1]

        chromosomes.append(child1)
        chromosomes.append(child2)

    for i in range(num_of_chromosomes, len(chromosomes)):
        tmp = fitness_function(chromosomes[i], edges_weight)
        fitness_results.append((i, tmp))
    fitness_results.sort(key=lambda x: x[1])

    return fitness_results[0][1], chromosomes[fitness_results[0][0]]

In [12]:
salesman_benchmarks = {'a280': open('benchmarks_salesman/a280.tsp', 'r').readlines()[6:-1],
                           'att48': open('benchmarks_salesman/att48.tsp', 'r').readlines()[6:-1],
                           'bays29': open('benchmarks_salesman/bays29.tsp', 'r').readlines()[38:-1],
                           'ch150': open('benchmarks_salesman/ch150.tsp', 'r').readlines()[6:-1],
                           'fl417': open('benchmarks_salesman/fl417.tsp', 'r').readlines()[6:-1],
                           }

for benchmark in salesman_benchmarks:
    coord = []
    for line in salesman_benchmarks[benchmark]:
        coord.append((float(line.split()[1]), float(line.split()[2])))
    w = []
    for i in range(len(coord)):
        w.append([])
        for j in range(len(coord)):
            w[i].append(euclidean_distance(coord[i], coord[j]))
    answers = []
    timeExecuted = 0
    for i in range(100):
        start = time.time()
        answer = get_ans(w)
        timeExecuted += time.time() - start
        answers.append(answer)

    print(f"BENCHMARK {benchmark}")
    print(f"TIME {timeExecuted/100*1000}")
    bestResult = min(list(map(lambda x: x[1], answers)))
    print(f'BEST RESULT: {bestResult}')
    for i in answers:
        if i[1] == bestResult:
            print(f'PATH: {i[0]}')
            break

BENCHMARK a280
TIME 168.8417625427246
BEST RESULT: [5, 133, 162, 22, 169, 67, 134, 181, 75, 62, 56, 50, 271, 220, 244, 73, 115, 173, 11, 9, 125, 132, 242, 144, 147, 137, 124, 179, 84, 47, 26, 98, 0, 240, 268, 76, 275, 270, 262, 83, 30, 266, 230, 74, 118, 141, 63, 78, 149, 79, 221, 251, 27, 104, 34, 23, 36, 29, 45, 81, 253, 194, 42, 97, 180, 157, 101, 191, 123, 121, 12, 256, 229, 40, 246, 198, 114, 235, 139, 28, 105, 53, 91, 213, 197, 188, 185, 161, 272, 274, 261, 200, 68, 215, 174, 66, 167, 182, 249, 172, 189, 217, 135, 146, 92, 168, 54, 170, 190, 163, 152, 279, 3, 278, 102, 46, 94, 228, 4, 267, 216, 2, 178, 193, 140, 65, 131, 82, 72, 129, 60, 184, 64, 223, 59, 171, 155, 241, 250, 87, 166, 148, 32, 204, 192, 164, 259, 55, 111, 96, 236, 106, 263, 248, 10, 16, 211, 150, 69, 80, 103, 222, 234, 238, 61, 127, 21, 214, 130, 116, 117, 258, 219, 245, 254, 224, 183, 77, 226, 31, 24, 89, 151, 13, 43, 57, 208, 153, 51, 119, 177, 252, 165, 107, 158, 70, 108, 112, 95, 218, 210, 237, 44, 25, 175, 7,

In [118]:
a = np.array([[1, 2, 3], [4, 5, 6], [7,8,9]])
b = random.choices(a, weights=[100,1,1], k=1)
b

[array([1, 2, 3])]

In [46]:
salesman_benchmarks = {'a280': open('benchmarks_salesman/a280.tsp', 'r').readlines()[6:-1],
                           'att48': open('benchmarks_salesman/att48.tsp', 'r').readlines()[6:-1],
                           'bays29': open('benchmarks_salesman/bays29.tsp', 'r').readlines()[38:-1],
                           'ch150': open('benchmarks_salesman/ch150.tsp', 'r').readlines()[6:-1],
                           'fl417': open('benchmarks_salesman/fl417.tsp', 'r').readlines()[6:-1],
                           }

for benchmark in salesman_benchmarks:
    coord = []
    for line in salesman_benchmarks[benchmark]:
        coord.append((float(line.split()[1]), float(line.split()[2])))
    w = []
    for i in range(len(coord)):
        w.append([])
        for j in range(len(coord)):
            w[i].append(euclidean_distance(coord[i], coord[j]))
    answers = []
    timeExecuted = 0
    for i in range(1):
        start = time.time()
        answer = genetic_salesman(w)
        timeExecuted += time.time() - start
        answers.append(answer)

    print(answers)
#     print(f"BENCHMARK {benchmark}")
#     print(f"TIME {timeExecuted/100*1000}")
#     bestResult = min(list(map(lambda x: x[1], answers)))
#     print(f'BEST RESULT: {bestResult}')
#     for i in answers:
#         if i[1] == bestResult:
#             print(f'PATH: {i[0]}')
#             break

86 147
[[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 127, 148, 232, 103, 12, 71, 224, 93, 187, 107, 183, 157, 28, 96, 57, 184, 185, 125, 115, 9, 2, 182, 41, 79, 135, 178, 62, 258, 221, 145, 86, 253, 122, 260, 102, 153, 226, 58, 202, 1, 186, 205, 25, 0, 244, 56, 192, 170, 200, 68, 39, 175, 10, 169, 20, 100, 46, 238, 161, 204, 37, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -

KeyboardInterrupt: 